# Sección 2. Entendimiento y Preparación de los datos

Esta sección tiene como propósito exponer el entendimiento y la preparación de los textos para posteriormente utilizarlos en la implementación de los modelos de aprendizaje. Para la manipulación de los datos se usarán librerias como **pandas** y **numpy**. Para la visualización se utilizarán **matplotlib**, **seaborn** y **WordCloud**.

## 1. Instalación e importación de librerias

### Instalación

In [2]:
#%pip install pandas numpy matplotlib seaborn scikit-learn nltk spacy wordcloud tqdm
#%pip install spacy-langdetect
#!python -m spacy download es_core_news_sm
#%pip install tensorflow
#%pip install keras
#!pip install wordcloud
#!pip install spacy-langdetect
#!python -m spacy download es_core_news_sm
#!pip install tensorflow
#!pip install keras
#!pip install spacy
#

In [3]:
# Manipulación de datos
import pandas as pd
import numpy as np

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# Procesamiento de texto
import re
import string
import spacy
from spacy_langdetect import LanguageDetector
import re
import unicodedata

import nltk
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords

# Modelado y evaluación
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, recall_score, precision_score, f1_score, ConfusionMatrixDisplay,precision_recall_curve


# Para búsqueda de hiperparámetros




from collections import Counter
#from ydata_profiling import ProfileReport
from scipy.sparse import hstack

import numpy as np


# Cargar modelo en español de spaCy
nlp = spacy.load("es_core_news_sm")

# Descargar stopwords de NLTK en español
nltk.download('stopwords')
stopwords_es = set(stopwords.words('spanish'))

# Configuración de visualización
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (10,5)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mgs05\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 2. Perfilamiento y entendimiento de los datos

#### 3.1. Limpieza de datos.

Este código implementa un proceso de preprocesamiento de texto para limpiar datos textuales antes de realizar análisis de texto o procesamiento de lenguaje natural.El preprocesamiento se aplica a dos columnas de un DataFrame (Titulo y Descripcion), generando nuevas columnas con los textos limpios (Titulo_Limpio y Descripcion_Limpia). Este paso es fundamental porque ayuda a reducir el ruido en los datos, mejora la precisión de los modelos y facilita la extracción de información relevante eliminando términos irrelevantes o redundantes.


In [4]:
# Cargar datos
data=pd.read_csv('./deception/data/fake_news_spanish.csv', sep=';', encoding = "utf-8")
news_df = data.copy()

In [5]:
import nltk
import unicodedata
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

news_df = news_df[["Label", "Titulo", "Descripcion"]]

# Funciones de preprocesamiento
def remove_non_ascii(words):
    """Remueve caracteres no ASCII de una lista de palabras"""
    return [unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore') for word in words if word]

def to_lowercase(words):
    """Convierte todas las palabras a minúsculas"""
    return [word.lower() for word in words if word]

def remove_punctuation(words):
    """Elimina signos de puntuación"""
    return [re.sub(r'[^\w\s]', '', word) for word in words if re.sub(r'[^\w\s]', '', word) != '']

def remove_numbers(words):
    """Elimina los números de la lista de palabras"""
    return [word for word in words if not word.isdigit()]

def remove_stopwords(words):
    """Elimina stopwords en español"""
    return [word for word in words if word not in stopwords_es]

def preprocessing(text):
    """Aplica todas las funciones de limpieza de texto a un string"""
    words = word_tokenize(text, language="spanish")  # Tokenizar con NLTK
    words = to_lowercase(words)
    words = remove_non_ascii(words)
    words = remove_punctuation(words)
    words = remove_numbers(words)
    words = remove_stopwords(words)
    return " ".join(words)  # Retorna el texto limpio como un string

# Aplicar limpieza en el DataFrame
news_df['Titulo_Limpio'] = news_df['Titulo'].astype(str).apply(preprocessing)
news_df['Descripcion_Limpia'] = news_df['Descripcion'].astype(str).apply(preprocessing)

# Comparación antes y después
news_df.head()


,Label,Titulo,Descripcion,Titulo_Limpio,Descripcion_Limpia
0,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,the guardian va sanchez europa necesita apuest...,diario britanico publico pasado jueves editori...
1,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,revelan gobierno negocio liberacion mireles ca...,revelan gobierno negocio liberacion mireles ca...
2,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,ahora nunca joan fuster estatuto valenciano cu...,valencianismo convoca castello fiesta grande c...
3,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",iglesias alienta yolanda diaz erc eh bildu neg...,politica igual negociar empresarios negociar g...
4,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",puigdemont seria ninguna tragedia repeticion e...,entrevista punt avui lider jxcat desdramatizad...


#### 3.2. Tokenización

Este código realiza la tokenización de texto. Esta tokenización se aplica a las columnas ya limpias del DataFrame (Titulo_Limpio y Descripcion_Limpia), generando nuevas columnas Titulo_Tokens y Descripcion_Tokens, donde cada texto se representa como una lista de palabras. Este paso es crucial porque permite a los modelos de análisis de texto procesar los datos de manera estructurada.


In [6]:
def tokenize_nltk(text):
    return word_tokenize(text, language="spanish")

# Aplicar tokenización en las columnas ya limpias
news_df['Titulo_Tokens'] = news_df['Titulo_Limpio'].astype(str).apply(tokenize_nltk)
news_df['Descripcion_Tokens'] = news_df['Descripcion_Limpia'].astype(str).apply(tokenize_nltk)

# Mostrar ejemplos
print("Comparación después de la tokenización:")
news_df.head()



Comparación después de la tokenización:


,Label,Titulo,Descripcion,Titulo_Limpio,Descripcion_Limpia,Titulo_Tokens,Descripcion_Tokens
0,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,the guardian va sanchez europa necesita apuest...,diario britanico publico pasado jueves editori...,"[the, guardian, va, sanchez, europa, necesita,...","[diario, britanico, publico, pasado, jueves, e..."
1,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,revelan gobierno negocio liberacion mireles ca...,revelan gobierno negocio liberacion mireles ca...,"[revelan, gobierno, negocio, liberacion, mirel...","[revelan, gobierno, negocio, liberacion, mirel..."
2,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,ahora nunca joan fuster estatuto valenciano cu...,valencianismo convoca castello fiesta grande c...,"[ahora, nunca, joan, fuster, estatuto, valenci...","[valencianismo, convoca, castello, fiesta, gra..."
3,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",iglesias alienta yolanda diaz erc eh bildu neg...,politica igual negociar empresarios negociar g...,"[iglesias, alienta, yolanda, diaz, erc, eh, bi...","[politica, igual, negociar, empresarios, negoc..."
4,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",puigdemont seria ninguna tragedia repeticion e...,entrevista punt avui lider jxcat desdramatizad...,"[puigdemont, seria, ninguna, tragedia, repetic...","[entrevista, punt, avui, lider, jxcat, desdram..."


#### 3.3. Lematización

Este código realiza la normalización del texto aplicando técnicas de stemming y lemmatización. La normalización se aplica a las columnas del DataFrame (Titulo_Tokens y Descripcion_Tokens), generando nuevas columnas Titulo_Normalizado y Descripcion_Normalizada, donde cada palabra es reducida a su raíz. Este paso es fundamental porque mejora la consistencia del texto y facilita el procesamiento de datos en modelos de análisis de texto.

In [7]:


# Descargar recursos necesarios para lematización
#nltk.download('wordnet')
#nltk.download('omw-1.4')

# Crear los objetos para stemming y lematización
stemmer = SnowballStemmer("spanish")
lemmatizer = WordNetLemmatizer()

def stem_words(words):
    """Aplica stemming para eliminar prefijos y sufijos en las palabras"""
    return [stemmer.stem(word) for word in words]

def lemmatize_verbs(words):
    """Aplica lematización para obtener la raíz de los verbos"""
    return [lemmatizer.lemmatize(word, wordnet.VERB) for word in words]

def stem_and_lemmatize(words):
    """Combina stemming y lematización para normalizar los datos"""
    stemmed = stem_words(words)
    lemmatized = lemmatize_verbs(words)
    return list(set(stemmed + lemmatized))  # Se usa set() para evitar duplicados

# Aplicar normalización a las columnas tokenizadas
news_df['Titulo_Normalizado'] = news_df['Titulo_Tokens'].apply(stem_and_lemmatize)
news_df['Descripcion_Normalizada'] = news_df['Descripcion_Tokens'].apply(stem_and_lemmatize)

# Mostrar ejemplos
print("Comparación después de la normalización:")
news_df.head()


Comparación después de la normalización:


,Label,Titulo,Descripcion,Titulo_Limpio,Descripcion_Limpia,Titulo_Tokens,Descripcion_Tokens,Titulo_Normalizado,Descripcion_Normalizada
0,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,the guardian va sanchez europa necesita apuest...,diario britanico publico pasado jueves editori...,"[the, guardian, va, sanchez, europa, necesita,...","[diario, britanico, publico, pasado, jueves, e...","[necesit, europ, frut, va, apuesta, europa, sa...","[adelanto, alerte, valiente, britanico, apoya,..."
1,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,revelan gobierno negocio liberacion mireles ca...,revelan gobierno negocio liberacion mireles ca...,"[revelan, gobierno, negocio, liberacion, mirel...","[revelan, gobierno, negocio, liberacion, mirel...","[gobiern, revelan, cambi, javi, negoci, liber,...","[armamento, trato, lanz, injusticias, salir, c..."
2,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,ahora nunca joan fuster estatuto valenciano cu...,valencianismo convoca castello fiesta grande c...,"[ahora, nunca, joan, fuster, estatuto, valenci...","[valencianismo, convoca, castello, fiesta, gra...","[fuster, joan, nunca, cumple, estatuto, ahor, ...","[valencianismo, fiesta, estatut, plana, plan, ..."
3,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",iglesias alienta yolanda diaz erc eh bildu neg...,politica igual negociar empresarios negociar g...,"[iglesias, alienta, yolanda, diaz, erc, eh, bi...","[politica, igual, negociar, empresarios, negoc...","[bildu, yolanda, erc, bloque, negoci, investid...","[igual, empresarios, grup, iglesi, reflexionad..."
4,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",puigdemont seria ninguna tragedia repeticion e...,entrevista punt avui lider jxcat desdramatizad...,"[puigdemont, seria, ninguna, tragedia, repetic...","[entrevista, punt, avui, lider, jxcat, desdram...","[repeticion, tragedia, elecciones, seria, ning...","[invest, elecciones, jxcat, investidura, desdr..."


Se mostrarán solamente: Los tokens del título, los tokens de la descripción, el titulo normalizado y la descripción normalizada.

In [8]:
processed_df = news_df[['Label', 'Titulo_Tokens', 'Descripcion_Tokens', 'Titulo_Normalizado', 'Descripcion_Normalizada']]
processed_df.head()

,Label,Titulo_Tokens,Descripcion_Tokens,Titulo_Normalizado,Descripcion_Normalizada
0,1,"[the, guardian, va, sanchez, europa, necesita,...","[diario, britanico, publico, pasado, jueves, e...","[necesit, europ, frut, va, apuesta, europa, sa...","[adelanto, alerte, valiente, britanico, apoya,..."
1,0,"[revelan, gobierno, negocio, liberacion, mirel...","[revelan, gobierno, negocio, liberacion, mirel...","[gobiern, revelan, cambi, javi, negoci, liber,...","[armamento, trato, lanz, injusticias, salir, c..."
2,1,"[ahora, nunca, joan, fuster, estatuto, valenci...","[valencianismo, convoca, castello, fiesta, gra...","[fuster, joan, nunca, cumple, estatuto, ahor, ...","[valencianismo, fiesta, estatut, plana, plan, ..."
3,1,"[iglesias, alienta, yolanda, diaz, erc, eh, bi...","[politica, igual, negociar, empresarios, negoc...","[bildu, yolanda, erc, bloque, negoci, investid...","[igual, empresarios, grup, iglesi, reflexionad..."
4,0,"[puigdemont, seria, ninguna, tragedia, repetic...","[entrevista, punt, avui, lider, jxcat, desdram...","[repeticion, tragedia, elecciones, seria, ning...","[invest, elecciones, jxcat, investidura, desdr..."


Ahora unificaremos los titulos y las descripciones normalizadas.

In [9]:
# Hacer copia del dataframe procesado
processed_news_df = processed_df.copy()

# Convertir las listas de palabras en texto separado por espacios en las columnas Titulo_Normalizado y Descripcion_Normalizada
processed_news_df['Titulo_Normalizado'] = processed_news_df['Titulo_Normalizado'].apply(lambda tokens: " ".join(tokens))
processed_news_df['Descripcion_Normalizada'] = processed_news_df['Descripcion_Normalizada'].apply(lambda tokens: " ".join(tokens))

# Mostrar algunos ejemplos para verificar el cambio
processed_news_df.head()

,Label,Titulo_Tokens,Descripcion_Tokens,Titulo_Normalizado,Descripcion_Normalizada
0,1,"[the, guardian, va, sanchez, europa, necesita,...","[diario, britanico, publico, pasado, jueves, e...",necesit europ frut va apuesta europa sanchez g...,adelanto alerte valiente britanico apoya elect...
1,0,"[revelan, gobierno, negocio, liberacion, mirel...","[revelan, gobierno, negocio, liberacion, mirel...",gobiern revelan cambi javi negoci liber duart ...,armamento trato lanz injusticias salir comienc...
2,1,"[ahora, nunca, joan, fuster, estatuto, valenci...","[valencianismo, convoca, castello, fiesta, gra...",fuster joan nunca cumple estatuto ahor anos es...,valencianismo fiesta estatut plana plan plen p...
3,1,"[iglesias, alienta, yolanda, diaz, erc, eh, bi...","[politica, igual, negociar, empresarios, negoc...",bildu yolanda erc bloque negoci investidura re...,igual empresarios grup iglesi reflexionado par...
4,0,"[puigdemont, seria, ninguna, tragedia, repetic...","[entrevista, punt, avui, lider, jxcat, desdram...",repeticion tragedia elecciones seria ningun ni...,invest elecciones jxcat investidura desdramati...


## 4. Construcción de Modelos

Como los datos que tenemos son datos de texto, debemos de alguna maner convertir estos datos en valores interpretables por el modelo para generar información relevante que podamos interpretar. En este caso, usaremos a nuestro favor la **vectorización de texto** a partir del método TF-IDF ya que es una opción balanceada entre simplicidad y efectividad. La idea es convertir los textos de 'Titulo_Normalizado' y 'Descripcion_Normalizada' en vectores numéricos usando TF-IDF y luego usarlos para entrenar un modelo de clasificación. 

#### 4.1 Transformación TF-IDF

In [10]:
# Definimos el vectorizador TF-IDF
vectorizer = TfidfVectorizer()  # Limitamos el número de características para evitar alta dimensionalidad

# Aplicamos la transformación TF-IDF a las columnas normalizadas
titulo_tfidf = vectorizer.fit_transform(processed_news_df['Titulo_Normalizado'])
descripcion_tfidf = vectorizer.fit_transform(processed_news_df['Descripcion_Normalizada'])

# Concatenamos ambas representaciones para tener una única matriz de características
X = hstack([titulo_tfidf, descripcion_tfidf])
# Definimos la variable objetivo
y = processed_news_df['Label']

# Mostramos la forma de la matriz final
print("Dimensión de la matriz de características X:", X.shape)
print("Dimensión de la variable objetivo y:", y.shape)

Dimensión de la matriz de características X: (57063, 91339)
Dimensión de la variable objetivo y: (57063,)


In [11]:
# AUXILIAR

# Aplicamos la transformación TF-IDF a las columnas normalizadas
titulo_tfidf = vectorizer.fit_transform(processed_news_df['Titulo_Normalizado'])
descripcion_tfidf = vectorizer.fit_transform(processed_news_df['Descripcion_Normalizada'])

# Concatenamos ambas representaciones para tener una única matriz de características
X = hstack([titulo_tfidf, descripcion_tfidf])

#### 4.2 Entrenamiento del primer modelo de clasificación - Regresión Logística

Utilizaremos un modelo clásico de Regresión Logística, ya que es simple, eficiente y suele funcionar bien en tareas de clasificación de texto.

In [12]:
# Dividimos los datos en entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Inicializamos y entrenamos el modelo de Regresión Logística
modelo = LogisticRegression(max_iter=1000)  # Aumentamos las iteraciones para asegurar convergencia
modelo.fit(X_train, y_train)

# Hacemos predicciones sobre el conjunto de prueba
y_pred = modelo.predict(X_test)

# Evaluamos el modelo con una matriz de confusión y métricas de desempeño
conf_matrix = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

#### 4.3 Resultados de la Regresión Logística

##### 4.3.1 Reporte de Clasificación

In [13]:
# Mostramos los resultados
print('Exactitud: %.2f' % accuracy_score(y_test, y_pred))
print("Recall: {}".format(recall_score(y_test,y_pred)))
print("Precisión: {}".format(precision_score(y_test,y_pred)))
print("Puntuación F1: {}".format(f1_score(y_test,y_pred)))

print("\nReporte de Clasificación:")
print(classification_rep)

Exactitud: 0.89
Recall: 0.9632086851628469
Precisión: 0.8637101135749053
Puntuación F1: 0.9107499287140006

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.94      0.79      0.86      4781
           1       0.86      0.96      0.91      6632

    accuracy                           0.89     11413
   macro avg       0.90      0.88      0.88     11413
weighted avg       0.90      0.89      0.89     11413



##### 4.3.2 Matríz de Confusión

##### 4.3.4 Modificación del Umbral de clasificación

**Análisis de la curva Presición-Recall en función del Umbral de Clasificación**

El punto de equilibrio lo encontramos entre 0.6 y 0.7 podemos empezar a experimentar con estos valores y evaluar los resultados.

In [15]:
# Definimos el nuevo umbral
nuevo_umbral = 0.6
y_prob = modelo.predict_proba(X_test)[:, 1] 
# Aplicamos el nuevo umbral a las predicciones
y_pred_ajustado = (y_prob >= nuevo_umbral).astype(int)

# Calculamos las nuevas métricas
from sklearn.metrics import  accuracy_score, classification_report

accuracy_ajustada = accuracy_score(y_test, y_pred_ajustado)
classification_rep_ajustado = classification_report(y_test, y_pred_ajustado)

print("\nReporte de Clasificación (Umbral 0.6):")
print(classification_rep_ajustado)
print(f"\nExactitud del modelo (Umbral 0.6): {accuracy_ajustada:.4f}")



Reporte de Clasificación (Umbral 0.6):
              precision    recall  f1-score   support

           0       0.89      0.84      0.87      4781
           1       0.89      0.93      0.91      6632

    accuracy                           0.89     11413
   macro avg       0.89      0.88      0.89     11413
weighted avg       0.89      0.89      0.89     11413


Exactitud del modelo (Umbral 0.6): 0.8911
